Script de select datos Cassandra en cluster multidomain

In [50]:
!pip install mysql-connector==2.1.7
!pip install pandas
!pip install sqlalchemy
#requiere instalación adicional, consultar https://github.com/PyMySQL/mysqlclient
!pip install mysqlclient
!pip install numpy
!pip install pymysql

In [123]:
import pandas as pd
import numpy as np
import os
import json
import random
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import time
from pprint import pprint
import psutil
import uuid
from cassandra.query import tuple_factory
from cassandra.query import dict_factory
from cassandra.query import BatchStatement, SimpleStatement
from cassandra.policies import RetryPolicy

In [124]:
#Los resultados de medidas de tiempo en carga por dominios se almacenan en estos objetos.
#Se itera durante 100 iteraciones para sacar medias
#repeticiones
repeats = 100

In [125]:
#Ficheros de salida
resultados_etl_select = '../Results/Cassandra/CassandraSelect_test_{}.csv'

In [126]:
def save_results_to_csv(results,file):
    #Guardamos los resultados en csv
    from datetime import datetime
    
    csv_df = pd.DataFrame(results, columns=['Registros', 'Tiempo', 'CPU','Memoria'])
    dia = datetime.now().strftime("%d%m%Y_%H_%M_%S")
    print(file.format(str(dia)))
    csv_df.to_csv(file.format(str(dia)))

In [127]:
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory
from cassandra import ConsistencyLevel

profile = ExecutionProfile(
    load_balancing_policy=WhiteListRoundRobinPolicy(['127.0.0.1']),
    retry_policy=DowngradingConsistencyRetryPolicy(),
    consistency_level=ConsistencyLevel.ALL,
    serial_consistency_level=ConsistencyLevel.LOCAL_SERIAL,
    request_timeout=3600,
    row_factory=tuple_factory
)
cluster = Cluster(execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()
print(session.execute("SELECT release_version FROM system.local").one())

session.execute('USE currentaccountkeyspace')

rows = session.execute('SELECT count(*) FROM currentaccount')

for row in rows:
    print(row) 

('4.0.3',)


ReadTimeout: Error from server: code=1200 [Coordinator node timed out waiting for replica nodes' responses] message="Operation timed out - received only 0 responses." info={'consistency': 'ALL', 'required_responses': 2, 'received_responses': 0}

# Select test multidomain

In [128]:
#Obtenemos los posibles valores de pais. Se iterará por ellos para cambiar en bucle los registros
session.execute('USE customerprofilekeyspace')
result = session.execute("""SELECT PartyId FROM customerprofilekeyspace.customerprofile limit 1000;""")
partyId_list= []
for partyId in result:
    partyId_list.append(partyId[0])
print(len(partyId_list))

1000


In [129]:
random.choice(partyId_list)

'c4accf9c-bfb5-11ec-b0d9-ff5473b2f02a'

In [130]:
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement
from cassandra import concurrent

registers = []
batch = BatchStatement(consistency_level=ConsistencyLevel.ALL)
account_id_list = []

#Cargas Masiva con Many
def selectCollectionCustomerProfile():
    
    SELECT_CURRENTACCOUNT_STMT = """SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = '{}';"""            
    SELECT_CUSTOMER_PROFILE_STMT = """SELECT * FROM customerprofilekeyspace.customerprofile WHERE partyid = '{}';"""    
    SELECT_POSITIONKEEPING_STMT = """SELECT * FROM positionkeepingkeyspace.positionkeeping WHERE accountid = '{}';"""            
    
    session = cluster.connect('customerprofilekeyspace')
    iter = 0;
    i = 1
        
    for i in range(0,repeats): 
        
        partyid = random.choice(partyId_list)
        
        time_inicial = time.time()        
        
        print(SELECT_CUSTOMER_PROFILE_STMT.format(partyid))
        result_cp = session.execute(SELECT_CUSTOMER_PROFILE_STMT.format(partyid))
        print(SELECT_CURRENTACCOUNT_STMT.format(partyid))
        result_ca = session.execute(SELECT_CURRENTACCOUNT_STMT.format(partyid))
        
        for res_cp in result_cp:
            print("RESULTADO CUSTOMER PROFILE", res_cp)
            
        for res_ca in result_ca:
            print("RESULTADO CURRENT ACCOUNT", res_ca)
            print(SELECT_POSITIONKEEPING_STMT.format(res_ca[0]))
            result_pk = session.execute(SELECT_POSITIONKEEPING_STMT.format(res_ca[0]))
            for res_pk in result_pk:
                print("RESULTADO POSITION_KEEPING", res_pk)                        
                time_final = time.time()           
                data_time_collection = round(time_final - time_inicial,3)
                used_cpu = psutil.cpu_percent()
                mem_used = psutil.virtual_memory().percent
                registers.append((iter,data_time_collection,used_cpu,mem_used))
                print((iter,data_time_collection,used_cpu,mem_used))
                iter += 1;
                time_inicial = time.time()
        i = i + 1
        
    return registers

In [131]:
registers = selectCollectionCustomerProfile()

SELECT * FROM customerprofilekeyspace.customerprofile WHERE partyid = 'c4ad3bbc-bfb5-11ec-b0d9-ff5473b2f02a';
SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = 'c4ad3bbc-bfb5-11ec-b0d9-ff5473b2f02a';
RESULTADO CUSTOMER PROFILE ('c4ad3bbc-bfb5-11ec-b0d9-ff5473b2f02a', 'DE.OBIE.Principal', '7', 'nboyle@example.org', 'Devyn Huel', 'Mr.', None, 'Marilou', '530', 'pri', '951.829.8107')
SELECT * FROM customerprofilekeyspace.customerprofile WHERE partyid = 'c4acf800-bfb5-11ec-b0d9-ff5473b2f02a';
SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = 'c4acf800-bfb5-11ec-b0d9-ff5473b2f02a';
RESULTADO CUSTOMER PROFILE ('c4acf800-bfb5-11ec-b0d9-ff5473b2f02a', 'UK.OBIE.Secundary', '8', 'dereck41@example.net', 'Montana Kessler', 'Ms.', None, 'Yoshiko', '9842', 'pri', '757.644.8385')
SELECT * FROM customerprofilekeyspace.customerprofile WHERE partyid = 'c4ad458a-bfb5-11ec-b0d9-ff5473b2f02a';
SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = 'c4ad458a-

SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = 'c4acc146-bfb5-11ec-b0d9-ff5473b2f02a';
RESULTADO CUSTOMER PROFILE ('c4acc146-bfb5-11ec-b0d9-ff5473b2f02a', 'DE.OBIE.Principal', '1', 'hintz.ayden@example.com', 'Amanda Wuckert', 'Miss', None, 'Thea', '7083', 'sol', '00397678421')
SELECT * FROM customerprofilekeyspace.customerprofile WHERE partyid = 'c4ad5ffc-bfb5-11ec-b0d9-ff5473b2f02a';
SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = 'c4ad5ffc-bfb5-11ec-b0d9-ff5473b2f02a';
RESULTADO CUSTOMER PROFILE ('c4ad5ffc-bfb5-11ec-b0d9-ff5473b2f02a', 'DE.OBIE.Principal', '8', 'nathanael76@example.net', 'Dr. Mabelle Pfannerstill', 'Dr.', None, 'Liza', '57', 'pri', '1-410-691-65')
SELECT * FROM customerprofilekeyspace.customerprofile WHERE partyid = 'c4ad9576-bfb5-11ec-b0d9-ff5473b2f02a';
SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = 'c4ad9576-bfb5-11ec-b0d9-ff5473b2f02a';
RESULTADO CUSTOMER PROFILE ('c4ad9576-bfb5-11ec-b0d9-ff5473b2f02a'

RESULTADO CUSTOMER PROFILE ('c4acc222-bfb5-11ec-b0d9-ff5473b2f02a', 'UK.OBIE.Principal', '2', 'oreilly@example.net', 'Abigale Funk Jr.', 'Ms.', None, 'Kristin', '3152', 'sol', '345-620-4050')
SELECT * FROM customerprofilekeyspace.customerprofile WHERE partyid = 'c4ad2942-bfb5-11ec-b0d9-ff5473b2f02a';
SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = 'c4ad2942-bfb5-11ec-b0d9-ff5473b2f02a';
RESULTADO CUSTOMER PROFILE ('c4ad2942-bfb5-11ec-b0d9-ff5473b2f02a', 'US.OBIE.Principal', '5', 'vweber@example.org', 'Unique Eichmann', 'Dr.', None, 'Barbara', '1072', 'sol', '05002184742')
SELECT * FROM customerprofilekeyspace.customerprofile WHERE partyid = 'c4acdd52-bfb5-11ec-b0d9-ff5473b2f02a';
SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = 'c4acdd52-bfb5-11ec-b0d9-ff5473b2f02a';
RESULTADO CUSTOMER PROFILE ('c4acdd52-bfb5-11ec-b0d9-ff5473b2f02a', 'UK.OBIE.Secundary', '4', 'zzieme@example.org', 'Miss Candice Boehm', 'Prof.', None, 'Loma', '4918', 'pri', '043289

RESULTADO CUSTOMER PROFILE ('c4ad30f4-bfb5-11ec-b0d9-ff5473b2f02a', 'UK.OBIE.Secundary', '5', 'myrtis.borer@example.net', 'Noelia Heller', 'Dr.', None, 'Jackie', '9747', 'pri', '743-829-3127')
SELECT * FROM customerprofilekeyspace.customerprofile WHERE partyid = 'c4ad00b6-bfb5-11ec-b0d9-ff5473b2f02a';
SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = 'c4ad00b6-bfb5-11ec-b0d9-ff5473b2f02a';
RESULTADO CUSTOMER PROFILE ('c4ad00b6-bfb5-11ec-b0d9-ff5473b2f02a', 'UK.OBIE.Secundary', '3', 'demond31@example.com', 'Sibyl Mertz', 'Mr.', None, 'Allene', '4467', 'pri', '139-062-5252')
SELECT * FROM customerprofilekeyspace.customerprofile WHERE partyid = 'c4ad70a0-bfb5-11ec-b0d9-ff5473b2f02a';
SELECT * FROM currentaccountkeyspace.currentaccount WHERE partyid = 'c4ad70a0-bfb5-11ec-b0d9-ff5473b2f02a';
RESULTADO CUSTOMER PROFILE ('c4ad70a0-bfb5-11ec-b0d9-ff5473b2f02a', 'US.OBIE.Principal', '6', 'nlueilwitz@example.com', 'Amaya Ullrich', 'Mrs.', None, 'Fausto', '5583', 'pri', '106-773

In [122]:
#Guardamos los resultados Customer Profile
save_results_to_csv(registers,resultados_etl_select)

../Results/Cassandra/CassandraSelect_test_19042022_02_47_24.csv


In [76]:
cluster.shutdown()
print('Conexion cerrada')

Conexion cerrada
